In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Runs multiple output commands in the same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
from sklearn.preprocessing import LabelEncoder

In [51]:
%run -i 'DataLoader.py'

In [52]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [53]:
train['item_cnt_day'] = train['item_cnt_day'].clip(0,20)

In [54]:
# Adding revenue now as it will be lost later when merged into monthly data
train['daily_revenue'] = train['item_price'] * train['item_cnt_day']

In [55]:
# Features to be extracted from date
train['date']=pd.to_datetime(train['date'],format="%d.%m.%Y")
train['month']=train['date'].dt.strftime("%m");
train['month']=train['month'].astype('int64');
train['year']=train['date'].dt.strftime("%Y");
train['year']=train['year'].astype('int64');
train['days_in_month'] = train['date'].dt.days_in_month
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,daily_revenue,month,year,days_in_month
0,2013-01-02,0,59,22154,999.00,1.0,999.00,1,2013,31
1,2013-01-03,0,25,2552,899.00,1.0,899.00,1,2013,31
2,2013-01-05,0,25,2552,899.00,0.0,0.00,1,2013,31
3,2013-01-06,0,25,2554,1709.05,1.0,1709.05,1,2013,31
4,2013-01-15,0,25,2555,1099.00,1.0,1099.00,1,2013,31
...,...,...,...,...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0,299.00,10,2015,31
2935845,2015-10-09,33,25,7460,299.00,1.0,299.00,10,2015,31
2935846,2015-10-14,33,25,7459,349.00,1.0,349.00,10,2015,31
2935847,2015-10-22,33,25,7440,299.00,1.0,299.00,10,2015,31


In [56]:
# Grouping train data into monthly data
monthly_data = train.groupby(['shop_id','item_id','date_block_num']).agg({'item_cnt_day':'sum','item_price':'mean','daily_revenue':'mean','month':'mean','year':'mean','days_in_month':'mean'}).reset_index(); # groupby automatically removes the column not mentioned, reset_index doesn't make the groupby columns as index
# monthly_data = monthly_data.rename(columns={'date_block_num':'month_block_num','item_price':'mean_item_price','item_cnt_day':'item_cnt_month','daily_revenue':'monthly_revenue'});

In [57]:
monthly_data['item_cnt_day'] = monthly_data['item_cnt_day'].clip(0,20)
monthly_data

,shop_id,item_id,date_block_num,item_cnt_day,item_price,daily_revenue,month,year,days_in_month
0,0,30,1,20.0,265.0,912.777778,2,2013,28
1,0,31,1,11.0,434.0,682.000000,2,2013,28
2,0,32,0,6.0,221.0,331.500000,1,2013,31
3,0,32,1,10.0,221.0,315.714286,2,2013,28
4,0,33,0,3.0,347.0,347.000000,1,2013,31
...,...,...,...,...,...,...,...,...,...
1609119,59,22164,27,2.0,699.0,699.000000,4,2015,30
1609120,59,22164,30,1.0,699.0,699.000000,7,2015,31
1609121,59,22167,9,1.0,299.0,299.000000,10,2013,31
1609122,59,22167,11,2.0,299.0,299.000000,12,2013,31


In [11]:
test

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [58]:
# We create test points in the train data set
date_block_df = pd.DataFrame(list(range(34)),columns=['date_block_num']);
date_block_df['month'] = date_block_df['date_block_num']%12+1
date_block_df['year'] = 2013+date_block_df['date_block_num']//12
month_days = pd.Series([0,31,28,31,30,31,30,31,31,30,31,30,31]);
date_block_df['days_in_month'] = date_block_df['month'].map(month_days)

date_block_df['key']=0;
test_copy = test.copy();
test_copy['key']=0;
test_copy = test_copy.merge(date_block_df);  # creates a cartesian product of all (shop, item) pairs in test will all date_block_num values
test_copy = test_copy.drop(['ID','key'],axis=1);
test_copy

,shop_id,item_id,date_block_num,month,year,days_in_month
0,5,5037,0,1,2013,31
1,5,5037,1,2,2013,28
2,5,5037,2,3,2013,31
3,5,5037,3,4,2013,30
4,5,5037,4,5,2013,31
...,...,...,...,...,...,...
7282795,45,969,29,6,2015,30
7282796,45,969,30,7,2015,31
7282797,45,969,31,8,2015,31
7282798,45,969,32,9,2015,30


In [59]:
# Merging this in train
monthly_data = pd.merge(test_copy,monthly_data,on=['shop_id','item_id','date_block_num','month','year','days_in_month'],how='left') 
monthly_data

In [12]:
# Making test like train and adding -1 to the columns that will be deleted later
test = test.drop(['ID'],axis=1)
test['date_block_num'] = 34;
test['item_cnt_day'] = -1;
test['item_price'] = -1;
test['daily_revenue'] = -1;
test['month'] = 11
test['year'] = 2015
test['days_in_month'] = 31
test

,shop_id,item_id,date_block_num,item_cnt_day,item_price,daily_revenue,month,year,days_in_month
0,5,5037,34,-1,-1,-1,11,2015,31
1,5,5320,34,-1,-1,-1,11,2015,31
2,5,5233,34,-1,-1,-1,11,2015,31
3,5,5232,34,-1,-1,-1,11,2015,31
4,5,5268,34,-1,-1,-1,11,2015,31
...,...,...,...,...,...,...,...,...,...
214195,45,18454,34,-1,-1,-1,11,2015,31
214196,45,16188,34,-1,-1,-1,11,2015,31
214197,45,15757,34,-1,-1,-1,11,2015,31
214198,45,19648,34,-1,-1,-1,11,2015,31


In [13]:
monthly_data = pd.concat([monthly_data, test], ignore_index=True, sort=False)

In [14]:
monthly_data

,shop_id,item_id,date_block_num,item_cnt_day,item_price,daily_revenue,month,year,days_in_month
0,0,30,1,20.0,265.0,912.777778,2,2013,28
1,0,31,1,11.0,434.0,682.000000,2,2013,28
2,0,32,0,6.0,221.0,331.500000,1,2013,31
3,0,32,1,10.0,221.0,315.714286,2,2013,28
4,0,33,0,3.0,347.0,347.000000,1,2013,31
...,...,...,...,...,...,...,...,...,...
1823319,45,18454,34,-1.0,-1.0,-1.000000,11,2015,31
1823320,45,16188,34,-1.0,-1.0,-1.000000,11,2015,31
1823321,45,15757,34,-1.0,-1.0,-1.000000,11,2015,31
1823322,45,19648,34,-1.0,-1.0,-1.000000,11,2015,31


In [15]:
# Text features

categories_translated['category_type'] = categories_translated['item_category_name_translated'].map(lambda x: x.split(' - ')[0])
categories_translated['category_sub-type'] = categories_translated['item_category_name_translated'].map(lambda x: x.split(' - ')[len(x.split(' - '))-1])

categories_translated['category_type_encoded'] = LabelEncoder().fit_transform(categories_translated['category_type'])
categories_translated['category_sub-type_encoded'] = LabelEncoder().fit_transform(categories_translated['category_sub-type'])

shops_translated['shop_city'] = shops_translated['shop_name_translated'].map(lambda x: x.split(' ')[0])
shops_translated['shop_city_encoded'] = LabelEncoder().fit_transform(shops_translated['shop_city'])

In [16]:
# Merging files
monthly_data = pd.merge(monthly_data,items,on=['item_id'],how='inner')
monthly_data = pd.merge(monthly_data,categories_translated,on=['item_category_id'],how='inner')
monthly_data = pd.merge(monthly_data,shops_translated,on=['shop_id'],how='inner')

monthly_data.head()

,shop_id,item_id,date_block_num,item_cnt_day,item_price,daily_revenue,month,year,days_in_month,item_name,item_category_id,item_category_name_translated,category_type,category_sub-type,category_type_encoded,category_sub-type_encoded,shop_name_translated,shop_city,shop_city_encoded
0,0,30,1,20.0,265.0,912.777778,2,2013,28,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,Cinema - DVD,Cinema,DVD,3,25,"Yakutsk Ordzhonikidze, 56 francs",Yakutsk,30
1,0,32,0,6.0,221.0,331.500000,1,2013,31,1+1,40,Cinema - DVD,Cinema,DVD,3,25,"Yakutsk Ordzhonikidze, 56 francs",Yakutsk,30
2,0,32,1,10.0,221.0,315.714286,2,2013,28,1+1,40,Cinema - DVD,Cinema,DVD,3,25,"Yakutsk Ordzhonikidze, 56 francs",Yakutsk,30
3,0,35,0,1.0,247.0,247.000000,1,2013,31,10 ЛЕТ СПУСТЯ,40,Cinema - DVD,Cinema,DVD,3,25,"Yakutsk Ordzhonikidze, 56 francs",Yakutsk,30
4,0,35,1,14.0,247.0,314.363636,2,2013,28,10 ЛЕТ СПУСТЯ,40,Cinema - DVD,Cinema,DVD,3,25,"Yakutsk Ordzhonikidze, 56 francs",Yakutsk,30


In [17]:
# Calculating Lagged features

def calculate_lag(lag,monthly_data,col):
    l=monthly_data.copy()
    l['date_block_num'] += lag
    l = l[['shop_id','item_id','date_block_num',col]]
    l = l.rename(columns={col:col+'_lag'+str(lag)});
    monthly_data = pd.merge(monthly_data,l,on=['shop_id','item_id','date_block_num'],how='left')
    return monthly_data

In [18]:
# Calculating Mean encoded features

def create_mean_features(train,monthly_data,groupby_cols,col_to_avg,new_col_name):
    cur_group = train.groupby(groupby_cols).agg({col_to_avg:'mean'}).reset_index();
    cur_group = cur_group.rename(columns={col_to_avg:new_col_name});
    monthly_data = pd.merge(monthly_data,cur_group, on = groupby_cols,how = 'left');
    return monthly_data

In [19]:
lag_window = 1;
for i in ['item_cnt_day','item_price','daily_revenue']:
    for j in range(0,lag_window):
        monthly_data = calculate_lag(j+1,monthly_data,i)
        
monthly_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1823324 entries, 0 to 1823323
Data columns (total 22 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   shop_id                        int64  
 1   item_id                        int64  
 2   date_block_num                 int64  
 3   item_cnt_day                   float64
 4   item_price                     float64
 5   daily_revenue                  float64
 6   month                          int64  
 7   year                           int64  
 8   days_in_month                  int64  
 9   item_name                      object 
 10  item_category_id               int64  
 11  item_category_name_translated  object 
 12  category_type                  object 
 13  category_sub-type              object 
 14  category_type_encoded          int64  
 15  category_sub-type_encoded      int64  
 16  shop_name_translated           object 
 17  shop_city                      object 
 18  sh

In [20]:
# Adding data to train to be used in calculating mean encoded features later
train = pd.merge(train,items,on=['item_id'],how='inner')
train = pd.merge(train,categories_translated,on=['item_category_id'],how='inner')
train = pd.merge(train,shops_translated,on=['shop_id'],how='inner')
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,daily_revenue,month,year,days_in_month,item_name,item_category_id,item_category_name_translated,category_type,category_sub-type,category_type_encoded,category_sub-type_encoded,shop_name_translated,shop_city,shop_city_encoded
0,2013-01-02,0,59,22154,999.0,1.0,999.0,1,2013,31,ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray,Cinema,Blu-Ray,3,9,"Yaroslavl Shopping center ""Altair""",Yaroslavl,31
1,2013-04-26,3,59,944,150.0,1.0,150.0,4,2013,30,2012 (BD),37,Cinema - Blu-Ray,Cinema,Blu-Ray,3,9,"Yaroslavl Shopping center ""Altair""",Yaroslavl,31
2,2013-06-26,5,59,944,199.5,1.0,199.5,6,2013,30,2012 (BD),37,Cinema - Blu-Ray,Cinema,Blu-Ray,3,9,"Yaroslavl Shopping center ""Altair""",Yaroslavl,31
3,2013-07-20,6,59,944,199.5,1.0,199.5,7,2013,31,2012 (BD),37,Cinema - Blu-Ray,Cinema,Blu-Ray,3,9,"Yaroslavl Shopping center ""Altair""",Yaroslavl,31
4,2013-09-14,8,59,944,299.0,2.0,598.0,9,2013,30,2012 (BD),37,Cinema - Blu-Ray,Cinema,Blu-Ray,3,9,"Yaroslavl Shopping center ""Altair""",Yaroslavl,31


In [21]:
# Mean encoded features

# Average of overall items sold in a month
monthly_data = create_mean_features(train,monthly_data,['date_block_num'],'item_cnt_day','item_cnt_avg_month')
monthly_data = calculate_lag(1,monthly_data,'item_cnt_avg_month')
# Average of overall items per item sold in a month
monthly_data = create_mean_features(train,monthly_data,['date_block_num','item_id'],'item_cnt_day','item_cnt_avg_item_month')
monthly_data = calculate_lag(1,monthly_data,'item_cnt_avg_item_month')
# Average of overall items per shop sold in a month
monthly_data = create_mean_features(train,monthly_data,['date_block_num','shop_id'],'item_cnt_day','item_cnt_avg_shop_month')
monthly_data = calculate_lag(1,monthly_data,'item_cnt_avg_shop_month')
# Average of overall items per category sold in a month
monthly_data = create_mean_features(train,monthly_data,['date_block_num','item_category_id'],'item_cnt_day','item_cnt_avg_category_month')
monthly_data = calculate_lag(1,monthly_data,'item_cnt_avg_category_month')
# Average of overall items per category type sold in a month
monthly_data = create_mean_features(train,monthly_data,['date_block_num','category_type'],'item_cnt_day','item_cnt_avg_categorytype_month')
monthly_data = calculate_lag(1,monthly_data,'item_cnt_avg_categorytype_month')
# Average of overall items per category subtype sold in a month
monthly_data = create_mean_features(train,monthly_data,['date_block_num','category_sub-type'],'item_cnt_day','item_cnt_avg_categorysubtype_month')
monthly_data = calculate_lag(1,monthly_data,'item_cnt_avg_categorysubtype_month')
# Average of overall items per shop city sold in a month
monthly_data = create_mean_features(train,monthly_data,['date_block_num','shop_city'],'item_cnt_day','item_cnt_avg_shopcity_month')
monthly_data = calculate_lag(1,monthly_data,'item_cnt_avg_shopcity_month')

monthly_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1823324 entries, 0 to 1823323
Data columns (total 36 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   shop_id                                  int64  
 1   item_id                                  int64  
 2   date_block_num                           int64  
 3   item_cnt_day                             float64
 4   item_price                               float64
 5   daily_revenue                            float64
 6   month                                    int64  
 7   year                                     int64  
 8   days_in_month                            int64  
 9   item_name                                object 
 10  item_category_id                         int64  
 11  item_category_name_translated            object 
 12  category_type                            object 
 13  category_sub-type                        object 
 14  category_type_enco

In [22]:
# #Months since last sale
monthly_data=monthly_data.sort_values(by=['shop_id','item_id','date_block_num'])
monthly_data['#months_since_last_sales'] = monthly_data['date_block_num']-monthly_data['date_block_num'].shift(1,axis=0)
monthly_data.loc[(monthly_data['shop_id']!=monthly_data['shop_id'].shift(1)) | (monthly_data['item_id']!=monthly_data['item_id'].shift(1)),'#months_since_last_sales']=0

monthly_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1823324 entries, 0 to 1729621
Data columns (total 37 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   shop_id                                  int64  
 1   item_id                                  int64  
 2   date_block_num                           int64  
 3   item_cnt_day                             float64
 4   item_price                               float64
 5   daily_revenue                            float64
 6   month                                    int64  
 7   year                                     int64  
 8   days_in_month                            int64  
 9   item_name                                object 
 10  item_category_id                         int64  
 11  item_category_name_translated            object 
 12  category_type                            object 
 13  category_sub-type                        object 
 14  category_type_enco

In [30]:
# Dropping lag window values
learning_data = monthly_data[monthly_data['date_block_num'] < lag_window]

# Features to delete
features_to_drop = ['item_price','daily_revenue','item_name','item_category_name_translated','category_type','category_sub-type','shop_name_translated','shop_city','item_cnt_avg_month','item_cnt_avg_item_month','item_cnt_avg_shop_month','item_cnt_avg_category_month','item_cnt_avg_categorytype_month','item_cnt_avg_categorysubtype_month','item_cnt_avg_shopcity_month'];
learning_data = monthly_data.drop(features_to_drop, axis=1);
learning_data = learning_data.rename(columns={'item_cnt_day':'item_cnt_month'});
learning_data.fillna(0,inplace=True)

learning_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1823324 entries, 0 to 1729621
Data columns (total 22 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   shop_id                                  int64  
 1   item_id                                  int64  
 2   date_block_num                           int64  
 3   item_cnt_month                           float64
 4   month                                    int64  
 5   year                                     int64  
 6   days_in_month                            int64  
 7   item_category_id                         int64  
 8   category_type_encoded                    int64  
 9   category_sub-type_encoded                int64  
 10  shop_city_encoded                        int64  
 11  item_cnt_day_lag1                        float64
 12  item_price_lag1                          float64
 13  daily_revenue_lag1                       float64
 14  item_cnt_avg_month